In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import numpy
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from collections import OrderedDict
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb

# %config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
# %matplotlib inline

In [2]:
train = pd.read_csv('../normalized_train.csv',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,

            },
        parse_dates=['fecha'])
test = pd.read_csv('../normalized_test.csv',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,

            },
        parse_dates=['fecha'])
train_raw = pd.read_csv('../train.csv',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,
                'tipodepropiedad': 'category',
                'provincia': 'category',
                'ciudad': 'category',

            },
        parse_dates=['fecha'])
test = test.set_index('id').dropna(subset=['titulo'])
train = train.set_index('id').dropna(subset=['titulo'])
df_all = train.append(test)
train_raw.shape

(240000, 23)

In [3]:
tipodepopiedades = [item.lower() for item in train_raw.tipodepropiedad.cat.categories]
tipodepopiedades

['apartamento',
 'bodega comercial',
 'casa',
 'casa en condominio',
 'casa uso de suelo',
 'departamento compartido',
 'duplex',
 'edificio',
 'huerta',
 'inmuebles productivos urbanos',
 'local comercial',
 'local en centro comercial',
 'lote',
 'nave industrial',
 'oficina comercial',
 'otros',
 'quinta vacacional',
 'rancho',
 'terreno',
 'terreno comercial',
 'terreno industrial',
 'villa',
 'hospedaje',
 'garage']

In [4]:
import nltk
import re
from unicodedata import normalize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
#nltk.download('stopwords')
stopwords = stopwords.words('spanish')


from collections import Counter
counter = Counter()
tokenizer = RegexpTokenizer(r'\w+')
a,b = 'áéíóúü','aeiouu'
trans = str.maketrans(a,b)

def countplease(x):
    
#     print(x)
    if(x!=x): return;
    
    x = tokenizer.tokenize(x)

    for word in list(x):  # iterating on a copy since removing will mess things up
        if word in stopwords or word.isnumeric() or word in tipodepopiedades:
            x.remove(word)

    x = [item.translate(trans) for item in x]
    counter.update(x)
    

df_all['titulo'].apply(lambda x: countplease(x))
counter.most_common(20)


[('venta', 132690),
 ('departamento', 47720),
 ('san', 20543),
 ('excelente', 15632),
 ('lomas', 13890),
 ('col', 13448),
 ('residencial', 12570),
 ('hermosa', 11738),
 ('condominio', 10751),
 ('valle', 9904),
 ('recamaras', 8851),
 ('oportunidad', 8725),
 ('santa', 8598),
 ('fracc', 8115),
 ('fraccionamiento', 7559),
 ('nueva', 7399),
 ('casas', 7364),
 ('remate', 7127),
 ('bonita', 6797),
 ('cerca', 6441)]

In [5]:
soloprecio = train[['logprecio', 'metros']]
solopreciotrain = soloprecio;
solotest = test[['metros']]
def buildDataframe(appearences, counter):
    newDict = dict()
    # Iterate over all the items in dictionary and filter items which has even keys
    for (key, value) in dict(counter).items():
        # Check if key is even then add pair to new dictionary
    #     print(value)
        if int(value) > appearences:
            newDict[key] = value

    # print('Filtered Dictionary : ')
    print(len(newDict.keys()))
    columns = list(newDict.keys())


    for column in columns: 
        soloprecio[column] = train['titulo'].str.contains(column)
        solotest[column] = test['titulo'].str.contains(column)
    #     columncontent = []
    return int(len(newDict.keys()))


In [6]:
clf = MLPRegressor(hidden_layer_sizes=(10), activation='tanh', solver='adam')

In [7]:
from sklearn.model_selection import GridSearchCV
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test_data)
    else:
      pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [8]:
from sklearn.model_selection import cross_val_score
columns = []
def rmse_cv(model, appearence):
    columns.append(buildDataframe(appearence, counter))

    
    every_column_except_y= [col for col in solopreciotrain.columns if col not in ['id', 'logprecio']]
    X_train, X_test, y_train, y_test = train_test_split(solopreciotrain[every_column_except_y], solopreciotrain.logprecio, test_size=0.2, random_state=123)
    print(solopreciotrain.logprecio.isnull().sum())
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = 2))
    return(rmse)

def xgBoost_rmse(model, appearence):
    columns.append(buildDataframe(appearence, counter))
    
    every_column_except_y= [col for col in solopreciotrain.columns if col not in ['id', 'logprecio']]
    X_train, X_test, y_train, y_test = train_test_split(solopreciotrain[every_column_except_y], solopreciotrain.logprecio, test_size=0.2, random_state=123)
    print(solopreciotrain.logprecio.isnull().sum())
    model.fit(X_train, y_train)
    
    print(np.sqrt(mean_squared_error(y_test, model.predict(X_test))))
    print(OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True)))
    return (OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True)))
#     return(rmse)

appearences = list(range(2000, 0, -200))
# cv_neuron = [rmse_cv(clf, appearence).mean() for appearence in appearences]

In [9]:
model = xgb.XGBRegressor(learning_rate =0.3, n_estimators=700, max_depth=5, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=6, scale_pos_weight=1, seed=27)
important_words = xgBoost_rmse(model, 600)

353


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[02:26:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.48765667857212114
OrderedDict([('metros', 4700), ('venta', 410), ('departamento', 328), ('d', 271), ('c', 266), ('depa', 259), ('m', 235), ('col', 205), ('san', 204), ('l', 198), ('b', 198), ('i', 190), ('f', 168), ('valle', 167), ('excelente', 154), ('n', 138), ('depto', 129), ('av', 127), ('centro', 118), ('sta', 116), ('residencial', 112), ('condominio', 111), ('local', 111), ('fracc', 108), ('comercial', 107), ('m2', 105), ('lomas', 101), ('fe', 101), ('pre', 96), ('min', 91), ('hermosa', 90), ('rec', 88), ('santa', 84), ('loma', 83), ('zona', 82), ('remate', 80), ('residencia', 80), ('real', 80), ('rio', 78), ('sur', 78), ('cerca', 77), ('casas', 77), ('departamentos', 76), ('bodega', 73), ('calle', 73), ('nuevo', 71), ('vista', 71), ('ciudad', 70), ('oportunidad', 69), ('colonia', 69), ('mar', 68), ('sol', 66), ('juriquilla', 65), ('gran', 65), ('quinta', 60), ('o

In [ ]:
# clffinal = MLPRegressor(activation='tanh', solver='adam')
# buildDataframe(15000)
# every_column_except_y= [col for col in solopreciotrain.columns if col not in ['id', 'logprecio']]
# X_train, X_test, y_train, y_test = train_test_split(solopreciotrain[every_column_except_y], solopreciotrain.logprecio, test_size=0.2, random_state=123)

In [ ]:
# cv_neuron = pd.Series(cv_neuron, index = columns)
# cv_neuron.plot(title = "Estimación solo con palabras del titulo")
# plt.xlabel("Columnas")
# plt.ylabel("rmse")
# cv_neuron

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(solopreciotrain[every_column_except_y], solopreciotrain.logprecio, test_size=0.2, random_state=123)
# np.sqrt(mean_squared_error(y_test, clf.predict(y_train)))

In [ ]:
clffinal = MLPRegressor(activation='tanh', solver='adam')
#buildDataframe(15000)
every_column_except_y= [col for col in solopreciotrain.columns if col not in ['id', 'logprecio']]
X_train, X_test, y_train, y_test = train_test_split(solopreciotrain[every_column_except_y], solopreciotrain.logprecio, test_size=0.2, random_state=123)

In [ ]:
# clffinal.fit(X_train, y_train)

In [11]:
# np.sqrt(mean_squared_error(y_test, clffinal.predict(X_test)))
len(important_words)

354

In [12]:
buildDataframe(20, Counter(important_words))

193


193

In [13]:
from sklearn.feature_extraction import FeatureHasher

cantidad_features = 30

h = FeatureHasher(n_features=cantidad_features, input_type='string')
every_column_except_y= [col for col in solopreciotrain.columns if col not in ['id', 'logprecio', 'metros']]

train_hashtrick = solopreciotrain.copy()
test_hashtrick = solotest.copy()

# Recorro las columnas y asigno la palabra si es True, sino nan.
for el in every_column_except_y:
    train_hashtrick.loc[train_hashtrick[el] == True, el] = el
    train_hashtrick.loc[train_hashtrick[el] == False, el] = np.nan
    test_hashtrick.loc[test_hashtrick[el] == True, el] = el
    test_hashtrick.loc[test_hashtrick[el] == False, el] = np.nan

# Armo la matriz de arrays para poder usar the hashing trick
train_hashtrick = train_hashtrick[every_column_except_y].apply(lambda x: list(filter(lambda y : y == y, x)), axis=1)
test_hashtrick = test_hashtrick[every_column_except_y].apply(lambda x: list(filter(lambda y : y == y, x)), axis=1)

# Termino de armar los arrays.
names = [f'fh{el + 1}' for el in range(cantidad_features)]
f = h.transform(train_hashtrick.values)
train_hashtrick = pd.DataFrame(f.toarray(), columns=names)
train_hashtrick['id'] = train.index
train_hashtrick = train_hashtrick.set_index('id')
f = h.transform(test_hashtrick.values)
test_hashtrick = pd.DataFrame(f.toarray(), columns=names)
test_hashtrick['id'] = test.index
test_hashtrick = test_hashtrick.set_index('id')
test_hashtrick

,fh1,fh2,fh3,fh4,fh5,fh6,fh7,fh8,fh9,fh10,...,fh21,fh22,fh23,fh24,fh25,fh26,fh27,fh28,fh29,fh30
id,,,,,,,,,,,,,,,,,,,,,
4941,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,-1.0,-1.0,1.0,-1.0,0.0,0.0,0.0,0.0,1.0
51775,0.0,-1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115253,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299321,0.0,-1.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173570,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75094,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
171847,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,-1.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0
138313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,-1.0


In [ ]:
test_hashtrick.shape

In [42]:
finaltest = pd.read_csv('../normalized_test.csv')
finaltest = finaltest.set_index('id')


test_raw = pd.read_csv('../test.csv',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,
                'tipodepropiedad': 'category',
                'provincia': 'category',
                'ciudad': 'category',

            },
        parse_dates=['fecha'])
idzonatest = test_raw.set_index('id')['idzona']
finaltest = finaltest.join(idzonatest, how='left')
finaltest = finaltest.join(test_hashtrick, how='left')
finaltest.loc[:, 'fecha'] = pd.to_datetime(finaltest.fecha).values.astype(np.int64)
finaltest.loc[:, names] = finaltest[names].fillna(0)
finaltest.to_csv('../normalized2_test.csv')
# finaltest.shape

In [43]:
finaltrain = pd.read_csv('../normalized_train.csv')
finaltrain = finaltrain.set_index('id')

idzonatest = train_raw.set_index('id')['idzona']
finaltrain = finaltrain.join(idzonatest, how='left')
finaltrain.loc[:, 'fecha'] = pd.to_datetime(finaltrain.fecha).values.astype(np.int64)
finaltrain = finaltrain.join(train_hashtrick)

finaltrain.loc[:, names] = finaltrain[names].fillna(0)
finaltrain.to_csv('../normalized2_train.csv')

In [34]:

# finaltrain.fecha.asType().values.astype(np.int64) // 10 ** 9


In [35]:
finaltrain.fecha


id
254099    1440288000000000000
53461     1372377600000000000
247984    1445040000000000000
209067    1331251200000000000
185997    1465257600000000000
                 ...         
119879    1423353600000000000
259178    1404950400000000000
131932    1425340800000000000
146867    1419552000000000000
121958    1434672000000000000
Name: fecha, Length: 239907, dtype: int64